In [4]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

In [5]:
load_dotenv(override=True)

True

In [6]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [7]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [8]:
import asyncio

In [13]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance with ComplAI

Dear [Recipient's Name],

I hope this message finds you well. I’m reaching out to introduce ComplAI, a leading SaaS solution designed to simplify SOC 2 compliance and audit preparation through the power of artificial intelligence.

Navigating the complexities of SOC 2 compliance can be a daunting task. ComplAI streamlines the process by automating documentation, tracking requirements, and ensuring that your team consistently meets compliance standards. Our platform is built to save you time, reduce overhead costs, and minimize audit-related stress.

Here are a few key benefits our clients have experienced:

- **Automated Documentation**: Effortlessly generate and manage compliance documents.
- **Real-Time Monitoring**: Stay on top of compliance requirements with instant updates.
- **Seamless Audits**: Accelerate the audit process with organized data and documentation.

I would welcome the opportunity to discuss how ComplAI can spe

In [14]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]
for output in outputs:
    print(output + "\n" + "-"*40 + "\n")

Subject: Simplify Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out on behalf of ComplAI—a leading provider of SaaS solutions designed to streamline SOC 2 compliance and audit preparations.

In today’s regulatory landscape, ensuring data security and maintaining compliance isn’t just a priority; it’s a necessity. However, the complexities of these processes can be daunting and time-consuming. That's where ComplAI comes in.

Our AI-powered platform not only simplifies documentation and compliance tracking but also provides real-time insights, helping you stay audit-ready with minimal effort. Whether you’re preparing for your first audit or seeking to refine your existing processes, ComplAI offers a tailored solution to meet your unique needs.

I’d love to schedule a quick call to discuss how ComplAI can enhance your compliance strategy and alleviate some of the burdens associated with SOC 2 requir

In [15]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [16]:
emails = "Cold sales emails:\n\n".join(outputs)
best = await Runner.run(sales_picker, emails)
print("Best cold sales email:\n")
print(best.final_output)
                                      

Best cold sales email:

Subject: 🚀 Your SOC 2 Compliance Just Met Its Match!

Hey [First Name],

You know what they say: shortcuts don’t exist when you’re preparing for a SOC 2 audit. But what if I told you that’s not entirely true?

At ComplAI, we’ve harnessed the power of AI to turn compliance from a “Nightmare on Compliance Street” into a delightful picnic filled with process gardens and audit-proof sandwiches! 🍔✨

With our tool, you’ll:
- Sail through audits like a pro (and not like a deer in headlights).
- Cut down on compliance chaos—goodbye endless spreadsheets!
- Sleep like a baby knowing you’re all set for whatever the auditors throw your way.

Interested in a quick chat? I promise I won’t make you sit through a terrifying horror movie! 👻 Just some fun banter about making compliance feel less “ugh” and more “woohoo!”

Hit me up if you’re ready to turn those compliance frowns upside down.

Best,  
[Your Name]  
[Your Title]  
ComplAI  
[Your Contact Info]  

P.S. If you can nam

## Use of Tool to send emails   

In [23]:
@function_tool
def send_email(body: str) -> str:
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("swapnilpuri@gmail.com")
    to_email = To("swapnilpuri@hotmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales Email", content)
    response = sg.client.mail.send.post(request_body=mail.get())
    return f"Email sent with status code {response.status_code}"

In [24]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001AE595056C0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

## Convert agent to tool

In [25]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="professional_sales_agent", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="engaging_sales_agent", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="busy_sales_agent", tool_description=description)

alltools = [tool1, tool2, tool3, send_email]

alltools

[FunctionTool(name='professional_sales_agent', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'professional_sales_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001AE5DA69F80>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='engaging_sales_agent', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'engaging_sales_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001AE5DA6ADE0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),


In [26]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=alltools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

### Handoffs: means agent delegating task to another agent and passing control to it.

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across

In [27]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [33]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("swapnilpuri@gmail.com")  
    to_email = To("swapnilpuri@hotmail.com")  
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [34]:
tools = [subject_tool, html_tool, send_html_email]

In [35]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


In [36]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='professional_sales_agent', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'professional_sales_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001AE5DA69F80>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='engaging_sales_agent', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'engaging_sales_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001AE5DA6ADE0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), F

In [37]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)